In [90]:
%run common_data_setup.ipynb

In [91]:
%run utils.ipynb

In [3]:
from yfpy import Data
from yfpy.logger import get_logger
from yfpy.query import YahooFantasySportsQuery

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json

[2023-12-29 15:45:33,950 DEBUG] [matplotlib.__init__.wrapper] matplotlib data path: C:\Users\mikem\anaconda3\Lib\site-packages\matplotlib\mpl-data
[2023-12-29 15:45:33,954 DEBUG] [matplotlib.__init__.wrapper] CONFIGDIR=C:\Users\mikem\.matplotlib
[2023-12-29 15:45:33,955 DEBUG] [matplotlib.__init__.<module>] interactive is False
[2023-12-29 15:45:33,955 DEBUG] [matplotlib.__init__.<module>] platform is win32
[2023-12-29 15:45:34,061 DEBUG] [matplotlib.__init__.wrapper] CACHEDIR=C:\Users\mikem\.matplotlib
[2023-12-29 15:45:34,065 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json
[2023-12-29 15:45:34,065 DEBUG] [matplotlib.font_manager.font_manager._load_fontmanager] Using fontManager instance from C:\Users\mikem\.matplotlib\fontlist-v330.json


In [4]:
league_ids

{'2015': 853925,
 '2016': 810238,
 '2017': 192551,
 '2018': 799346,
 '2019': 820083,
 '2020': 394682,
 '2021': 364325,
 '2022': 564704,
 '2023': 333146}

In [5]:
game_keys_df

,season,code
16,2015,348
17,2016,359
18,2017,371
19,2018,380
20,2019,390
21,2020,399
22,2021,406
23,2022,414
24,2023,423


In [88]:
yahoo_query = YahooFantasySportsQuery(
    auth_dir,
    league_id=853925,
    game_code='nfl',
    game_id=348
)
teams = yahoo_query.get_league_teams()
team_ids_list = []
for team in teams:
    team_name = str(team.name)[1:].strip("'")
    manager = team.managers[0].nickname
    id = team.team_id
    info = {'team_name': team_name, 'manager_nickname': manager, 'id': id}
    team_ids_list.append(info)
team_ids_df = pd.DataFrame(team_ids_list)

In [92]:
max_weeks = len(yahoo_query.get_game_info_by_game_id(348))
season_stats = []
for week in range(1, max_weeks+1):
    for manager_id in team_ids_df['id']:
        week_subquery = yahoo_query.get_team_roster_player_stats_by_week(manager_id, week)
        weekly_stats = generate_roster_points_by_week(week_subquery)
        weekly_stats['team_name'] = team_ids_df.loc[team_ids_df['id'] == manager_id, 'team_name'].item()
        weekly_stats['week'] = week
        season_stats.append(weekly_stats)

In [93]:
season_stats_df = pd.DataFrame(season_stats)

In [95]:
season_stats_df.to_csv('2015_weekly_roster_stats.csv', index=False)

In [87]:
team_ids_df.loc[team_ids_df['id'] == 11, 'team_name'].item()

'Thug Bungalow'

In [16]:
week = yahoo_query.get_team_roster_player_stats_by_week(11, 1)